# Load Libraries and Download Data

In [ ]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=0ce49a0b3122a20cd2b4b27398e1f1d9324be2a05270e3c98d276db351983ae5
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
!pip install torchtext==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.16.0
    Uninstalling torchtext-0.16.0:
      Successfully uninstalled torchtext-0.16.0


In [ ]:
import re
import string
import zipfile
import pandas as pd
import numpy as np
import nltk
import torch
import torch.nn.functional as F
import torchtext
from torchviz import make_dot
from torchsummary import summary

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator, LabelField, TabularDataset

import copy
import gc
import random
import sys
import xgboost as xgb
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score



from collections import defaultdict, Counter

import torch
import torch.nn as nn
torch.backends.cudnn.deterministic = True

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# IMDB REVIEWS
!pip install kaggle

In [ ]:
!pip install torch-summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Applied Machine Learning/Project Milestone 2/IMDB Dataset.csv.zip"
# !unzip "/content/drive/MyDrive/Applied Machine Learning/Project Milestone 2/glove.6B.50d.txt.zip"

Archive:  /content/drive/MyDrive/Applied Machine Learning/Project Milestone 2/IMDB Dataset.csv.zip
  inflating: IMDB Dataset.csv        


In [ ]:
reviews_df = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
filtered_reviews = reviews_df[reviews_df['review'].str.contains('Playmates')]
print(filtered_reviews['review'])
print(string.punctuation)

6629     H.O.T.S. is proof that at one time, the movie ...
13403    I gave this a 10 because it's the best film of...
14497    A Turkish Bath sequence in a film noir loc...
18844    Hello Playmates.I recently watched this film f...
Name: review, dtype: object
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


# Preprocessing

In [ ]:
# Preprocessing

def clean_string(x):
    # remove punctuations
    x = ''.join(' ' if char in string.punctuation else char for char in x) # replacing with a space because some reviews have no space after punctuation "I.like"
    x = re.sub('[^a-zA-Z0-9]', ' ', x) # remove non-alpha numeric
    x = re.sub('\s+', ' ', x) # remove more than one spaces
    x = x.lower().strip() # lower case trailing leading spaces
    return x

def remove_html_tags(x): # using regex misses cases like '&nsbm' or anchor tags like <a title=">">, lxml has <p> tag problem
  x = BeautifulSoup(x, "lxml").text
  return x

def remove_stopwords(x):
    stopwords_set = set(stopwords.words('english'))
    words = x.split()
    filtered_words = [word for word in words if word.lower() not in stopwords_set]
    return ' '.join(filtered_words)

def lemmatization(x):
  lemmatizer = WordNetLemmatizer()
  words = x.split()
  lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
  lemmatized_sentence = ' '.join(lemmatized_words)
  return lemmatized_sentence

In [ ]:
reviews_df_cleaned = reviews_df.copy()
reviews_df_cleaned['review'] = reviews_df['review'].apply(remove_html_tags)
reviews_df_cleaned['review'] = reviews_df_cleaned['review'].apply(clean_string)
reviews_df_cleaned['review'] = reviews_df_cleaned['review'].apply(remove_stopwords)
reviews_df_cleaned['review'] = reviews_df_cleaned['review'].apply(lemmatization)

<ipython-input-9-1c94d9d519d6>:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  x = BeautifulSoup(x, "lxml").text


In [ ]:
def sentiment_to_integer(x):
    if x == 'positive':
        return 1
    elif x == 'negative':
        return 0

reviews_df_cleaned['sentiment'] = reviews_df_cleaned['sentiment'].apply(sentiment_to_integer)
# print(reviews_df_cleaned)
# positive_reviews_count = reviews_df_cleaned[reviews_df_cleaned['sentiment'] == 1]['review'].count()
# # print(positive_reviews_count)
# negative_reviews_count = reviews_df[reviews_df_cleaned['sentiment'] == 0]['review'].count()
# negative_reviews_count

# Saving Preprocessed Dataset back to a CSV file

In [ ]:
reviews_df_cleaned.to_csv('/content/drive/MyDrive/Applied Machine Learning/Project Milestone 2/IMDBReviewsCleaned.csv', index=False)

In [ ]:
reviews_df_cleaned = pd.read_csv('/content/drive/MyDrive/Applied Machine Learning/Project Milestone 2/IMDBReviewsCleaned.csv')

reviews_df_cleaned

,review,sentiment
0,one reviewer mentioned watching 1 oz episode h...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake think zombie ...,0
4,petter mattei love time money visually stunnin...,1
...,...,...
49995,thought movie right good job creative original...,1
49996,bad plot bad dialogue bad acting idiotic direc...,0
49997,catholic taught parochial elementary school nu...,0
49998,going disagree previous comment side maltin on...,0


# Dataset Preparation before inputting  to RNN (Torchtext)

In [ ]:
# https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
# https://torchtext.readthedocs.io/en/latest/data.html

! pip install spacy
# to use tokenizer from known NLP library spacy -> efficeint for large datasets compared to traditional python methods + more robust.
# also provides support for tokenization in languages other than english.

In [ ]:
! python -m spacy download en_core_web_sm # downloading the english vocab

2023-12-01 17:33:05.180441: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 17:33:05.180511: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 17:33:05.180564: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 17:33:05.195215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 17:33:06.917797: W tensorflow/compiler/

In [ ]:
# text and label formatting

# TEXT will be our feature/token

TEXT = Field(
    tokenize='spacy',
    tokenizer_language='en_core_web_sm'
)

LABEL = LabelField(dtype=torch.long)
print(TEXT)

In [ ]:
parsed_data = TabularDataset(
    path='/content/drive/MyDrive/Applied Machine Learning/Project Milestone 2/IMDBReviewsCleaned.csv', format='csv',
    skip_header = True, fields=[('review', TEXT), ('sentiment', LABEL)]
)

In [ ]:
# vars(parsed_data.examples[0])

# Train Test Validation Split

In [ ]:
# https://pytorch.org/text/_modules/torchtext/data/dataset.html

SEED = 42

torch.manual_seed(SEED)
random.seed(SEED)

train_ratio = 0.75
val_ratio = 0.125
test_ratio = 0.125


train_data, val_data, test_data = parsed_data.split(
    split_ratio=[train_ratio, val_ratio, test_ratio],
    stratified=True,  # distribution of labels is maintained in each split
    strata_field='sentiment',  # field used for stratification
    random_state=random.getstate()
)

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(val_data)}")
print(f"Number of test examples: {len(test_data)}")

Number of training examples: 37500
Number of validation examples: 6250
Number of test examples: 6250


In [ ]:
print(vars(train_data.examples[0]))

{'review': ['show', 'drive', 'crazy', 'go', 'everything', 'family', 'even', 'intended', 'comedy', 'show', 'suppose', 'follow', 'dave', 'vicky', 'gold', 'michael', 'rapaport', 'anita', 'barone', 'raise', 'three', 'teenage', 'child', 'hilary', 'larry', 'mike', 'good', 'premise', 'comedy', 'yes', 'mean', 'good', 'show', 'think', 'ever', 'heard', 'cruder', 'talk', 'parent', 'child', 'vice', 'versa', 'talk', 'seems', 'show', 'control', 'child', 'parent', 'sex', 'know', 'sexual', 'intercourse', 'usually', 'subject', 'talked', 'teenager', 'often', 'brought', 'nearly', 'every', 'episode', 'point', 'one', 'episode', 'watching', 'involved', 'parent', 'giving', 'daughter', 'hilary', 'car', 'think', 'related', 'anyway', 'way', 'talk', 'parent', 'talk', 'hilary', 'like', 'slave', 'show', 'fails', 'comedy', 'level', 'laughed', 'show', 'numerous', 'time', 'attempted', 'watch', 'person', 'gave', 'film', 'animal', 'house', 'dumb', 'dumber', 'high', 'mark', 'michael', 'rapaport', 'good', 'actor', 'choos

# Hyperparams

In [ ]:
vocab_size = 20000
# learning_rate = 0.01
# learning_rate = 0.001
learning_rate = 0.0005
batch_size = 64
# num_epochs = 15
num_epochs = 5
device = torch.device('cuda')

embedding_dim = 25
# embeddings = 50 that use glove
# hidden_dim = 16
hidden_dim = 8
# dropout_prob = 0.6
dropout_prob = 0.2
weight_decay = 1e-4
num_classes = 2

In [ ]:
# Total models = 3(lr) * 2(epochs) * 2(embedding) * 2(hidden_dim) * 2(dropout_prob) * 2(optimizers) * 3(datasets) = 288 models

# Vocabulary

In [ ]:
TEXT.build_vocab(train_data, max_size=vocab_size, vectors="glove.6B.50d")

.vector_cache/glove.6B.zip: 862MB [02:39, 5.41MB/s]                           
100%|█████████▉| 399999/400000 [00:19<00:00, 20711.25it/s]


In [ ]:
TEXT.build_vocab(train_data, max_size=vocab_size) # only on training data
LABEL.build_vocab(train_data)

print(f'Vocab Size {len(TEXT.vocab)}') # 2002 with <unk> and <pad> tokens
print(f'Label Size {len(LABEL.vocab)}')
print(TEXT.vocab.freqs.most_common(20))

# RNN can handle arbitrary lenghts but padding is rewuired for padding sequences to same length in
# a given minibatch so we can store those in an array

Vocab Size 20002
Label Size 2
[('movie', 76850), ('film', 69759), ('one', 41390), ('like', 30748), ('time', 23478), ('good', 22353), ('character', 21165), ('story', 18864), ('even', 18490), ('get', 18362), ('would', 18270), ('make', 18165), ('see', 17974), ('really', 17302), ('well', 16004), ('scene', 15955), ('much', 14440), ('bad', 13843), ('people', 13628), ('great', 13528)]


In [ ]:
print(TEXT.vocab.itos[:10]) # tokens for first 10 elements

['<unk>', '<pad>', 'movie', 'film', 'one', 'like', 'time', 'good', 'character', 'story']


In [ ]:
print(TEXT.vocab.stoi['movie'])

2


# Dataloaders (without dataset shift)



In [ ]:
# BucketIterator will group batches such that sentences have similar lenght and reduces number of padding required.
# Having batches with similar length examples provides a lot of gain for recurrent models (RNN, GRU, LSTM)
# and transformers models (bert, roBerta, gpt2, xlnet, etc.) where padding will be minimal.

# https://gmihaila.github.io/tutorial_notebooks/pytorchtext_bucketiterator/

train_loader, val_loader, test_loader = BucketIterator.splits(
    (train_data, val_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.review),
    sort = False,
    device=device
)

In [ ]:
for batch in train_loader:
  print(batch.review.size())
  print(batch.sentiment.size())
  break

for batch in val_loader:
  print(batch.review.size())
  print(batch.sentiment.size())
  break

for batch in test_loader:
  print(batch.review.size())
  print(batch.sentiment.size())
  break

RuntimeError: ignored

# Dataloaders with Dataset Shift

In [ ]:
def dataset_shift(X, mean = None, std = None):
  lst = list()
  for i in range(len(X)):
    lst.append(len(X[i].split()))
  lst = np.array(lst)
  if mean is None:
    mean = np.mean(lst)
  if std is None:
    std = np.std(lst)
  len_short = round(mean - 1 / 3 * std)
  len_long = round(mean + 1 / 3 * std)
  X_short = list()
  X_long = list()
  for doc in X:
    document = doc.split()
    if len(document) < len_short:
      while len(document) != len_short:
        document.append('<PAD>')
      X_short.append(document)
    else:
      X_short.append(document[:len_short])
    document = doc.split()
    if len(document) < len_long:
      while len(document) != len_long:
        document.append('<PAD>')
      X_long.append(document)
    else:
      X_long.append(document[:len_long])
  return np.array(X_short), np.array(X_long), mean, std

In [ ]:
from torchtext.data import Example, Field, TabularDataset, BucketIterator, Dataset

def create_datasets_loaders(data, batch_size, device, mean=None, std=None):
    texts = [' '.join(example.review) for example in data.dataset.examples]
    X_short, X_long, mean, std = dataset_shift(texts, mean, std)

    fields = [('review', TEXT), ('sentiment', LABEL)]
    examples_short = [Example.fromlist([X_short[i], example.sentiment], fields) for i, example in enumerate(data.dataset.examples)]
    examples_long = [Example.fromlist([X_long[i], example.sentiment], fields) for i, example in enumerate(data.dataset.examples)]

    dataset_short = Dataset(examples=examples_short, fields=fields)
    dataset_long = Dataset(examples=examples_long, fields=fields)

    loader_short, loader_long = BucketIterator.splits(
        (dataset_short, dataset_long),
        batch_size=batch_size,
        sort_within_batch=True,
        sort_key=lambda x: len(x.review),
        sort = False,
        device=device
    )

    return loader_short, loader_long, mean, std

In [ ]:
train_loader_short, train_loader_long, mean, std = create_datasets_loaders(train_loader, batch_size, device)

In [ ]:
print(next(iter(train_loader)).review[0])
print(next(iter(train_loader_short)).review[0])

tensor([ 146,  410,   16,  444,   23,  118,  544,   67,    4,    2,   16,   29,
          10, 6799,  220,   21,  220, 6079,  310,  555,  101,   23,    4,  118,
         315,  178,  118,  670,    4,   15,  486, 2230,  707,  670, 6820,    2,
         317,  691,    3, 7574,   19,  613,    2,    0,  275, 1399,  571, 5353,
        2905, 2975,   44,  294, 2283,   41,  106,   46,    2,  118,  334, 2933,
           2,    2, 1674,    2], device='cuda:0')
tensor([  874,  3267,   106,     4,   637,   453,    24,  6590,     2,   118,
         3706,   817,  6540,  1269,  5196,   185,    73,  1220,   742,     2,
         1557,  4078,     4,   787,    21,  9201,  1188,     7,    39,   278,
          317,   324,  5038,     4,  6994,    38,    95,     4,   278,   721,
            2,  1837, 14568,     2,    36,   366,    20,   118,     4,  2877,
            2,   133,    89,   113,   686,    29,  3075,     6,  1677,    93,
            4,   280,  2179,   342], device='cuda:0')


In [ ]:
val_loader_short, val_loader_long, _, _ = create_datasets_loaders(val_loader, batch_size, device, mean, std)

In [ ]:
test_loader_short, test_loader_long, _, _  = create_datasets_loaders(test_loader, batch_size, device, mean, std)

# Model

In [ ]:
# DL Model Architecture for IMDB Reviews Classification
# https://sebastianraschka.com/pdf/lecture-notes/stat453ss21/L15_intro-rnn__slides.pdf --> slide 29 - 47

# https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html



class SentimentAnalysis(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, dropout_prob=0.5, use_glove = False):
        super(SentimentAnalysis, self).__init__()

        # embedding layer --> input tokens to vectors
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        if use_glove:
          self.embedding.weight.data.copy_(TEXT.vocab.vectors)

        # self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        # LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # dropout layer --> prevent overfitting
        self.dropout = nn.Dropout(dropout_prob)

        # self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # fc
        self.fc_hidden = nn.Linear(hidden_dim, hidden_dim)

        # output layer for final classification
        self.fc_output = nn.Linear(hidden_dim, output_dim)

    def forward(self, text): # [sentence_length, batch_size]
        embedded = self.embedding(text) # in embedding matrix: [sentence_length (rows), batch_size (columns), dimension of hidden layer]

        output, (hn, _) = self.lstm(embedded) # one lstm will output y, hidden state for next layer

        hn = self.dropout(hn) # using dropout for regularization

        # output from the last timestep to get context of the entire sequence
        last_hidden = (hn[0] + hn[1]) / 2
        # last_hidden = hn
        last_hidden.squeeze_(0) # so that we can send it to the linear layer ahead.
        # instead of taking final output from LSTM we are taking the last hidden layer and applying our own output function which is fc + relu

        hidden_output = F.relu(self.dropout(self.fc_hidden(last_hidden)))

        final_output = self.fc_output(hidden_output)
        return final_output

In [ ]:
len(TEXT.vocab)

20002

# Model Initialization

In [ ]:
torch.manual_seed(42)
model = SentimentAnalysis(input_dim = len(TEXT.vocab),
                          embedding_dim=embedding_dim,
                          hidden_dim=hidden_dim,
                          output_dim=num_classes,
                          dropout_prob = dropout_prob,
                          use_glove = True
)

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# # @title Testing model Initialization
# variable_name = ""
# variable_name = ""
# y_hat = model(torch.Tensor([[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]).to(torch.long).to(device))

In [ ]:
# y_hat

In [ ]:
# make_dot(y_hat, params=dict(list(model.named_parameters()))).render("rnn_torchviz", format="png")

In [ ]:
# print(model)

In [ ]:
gc.collect()

8

In [ ]:
# summary(model, (100, ), dtypes=[torch.long], verbose=2, col_width=16, col_names=["kernel_size", "output_size", "num_params", "mult_adds"],)

# Training Validation Test loop

In [ ]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():
        correct_pred, num_examples, num_batches = 0, 0, 0
        F1 = 0
        for i, (features, targets) in enumerate(data_loader):
            features = features.to(device)
            targets = targets.float().to(device)
            logits = F.softmax(model(features), dim =1)
            _, predicted_labels = torch.max(logits, 1)
            num_examples += targets.size(0)
            num_batches += 1
            F1 += f1_score(torch.Tensor.cpu(targets), torch.Tensor.cpu(predicted_labels))
            correct_pred += (predicted_labels == targets).sum()
    return float(correct_pred.float()/num_examples * 100), F1 / num_batches

In [ ]:
def train_val_loop_test(model, optimizer, num_epochs, device, train_loader_input, val_loader_input, test_loader_input):
  train_loader = train_loader_input
  val_loader = val_loader_input
  test_loader = test_loader_input
  train_scores = list()
  val_scores = list()
  train_losses = list()
  val_losses = list()
  for epoch in range(num_epochs):
      model.train()
      loss_agg = 0
      num_batches = 0
      for batch_idx, batch_data in enumerate(train_loader):

          text = batch_data.review.to(device)
          labels = batch_data.sentiment.to(device)

          logits = model(text)
          loss = F.cross_entropy(logits, labels)
          loss_agg += float(loss)
          num_batches += 1
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          # if not batch_idx % 50:
          #     print (f'Epoch: {epoch+1}/{num_epochs}' \
          #           f'Batch {batch_idx}/{len(train_loader)}' \
          #           f'Loss: {loss}')

      loss_val_agg = 0
      num_val_batches = 0
      for batch_idx, batch_data in enumerate(val_loader):

          text = batch_data.review.to(device)
          labels = batch_data.sentiment.to(device)

          logits = model(text)
          loss = F.cross_entropy(logits, labels)
          loss_val_agg += float(loss)
          num_val_batches += 1

      with torch.set_grad_enabled(False):
          print("Epoch:", epoch)
          print(f'training accuracy/f1: '
                f'{compute_accuracy(model, train_loader, device)}'
                f'\nvalid accuracy/f1: '
                f'{compute_accuracy(model, val_loader, device)}')
          train_losses.append(loss_agg/num_batches)
          val_losses.append(loss_val_agg / num_val_batches)
          train_scores.append(compute_accuracy(model, val_loader, device))
          val_scores.append(compute_accuracy(model, val_loader, device))

  print(f'Test accuracy/f1: {compute_accuracy(model, test_loader, device)}')
  return train_scores, val_scores, train_losses, val_losses

In [ ]:
train_val_loop_test(model, optimizer, num_epochs, device, train_loader, val_loader, test_loader)

Epoch: 1/5Batch 0/586Loss: 0.3645293414592743
Epoch: 1/5Batch 50/586Loss: 0.372294157743454
Epoch: 1/5Batch 100/586Loss: 0.2793301045894623
Epoch: 1/5Batch 150/586Loss: 0.4087273180484772
Epoch: 1/5Batch 200/586Loss: 0.49754753708839417
Epoch: 1/5Batch 250/586Loss: 0.39730966091156006
Epoch: 1/5Batch 300/586Loss: 0.17276531457901
Epoch: 1/5Batch 350/586Loss: 0.4726679027080536
Epoch: 1/5Batch 400/586Loss: 0.25318723917007446
Epoch: 1/5Batch 450/586Loss: 0.4106673300266266
Epoch: 1/5Batch 500/586Loss: 0.2871471345424652
Epoch: 1/5Batch 550/586Loss: 0.4723036587238312
training accuracy/f1: (87.68533325195312, 0.8752800091370858)
valid accuracy/f1: (83.5199966430664, 0.8300104193801696)
Epoch: 2/5Batch 0/586Loss: 0.32929641008377075
Epoch: 2/5Batch 50/586Loss: 0.5541124939918518
Epoch: 2/5Batch 100/586Loss: 0.3666008412837982
Epoch: 2/5Batch 150/586Loss: 0.19914399087429047
Epoch: 2/5Batch 200/586Loss: 0.2843019664287567
Epoch: 2/5Batch 250/586Loss: 0.31596535444259644
Epoch: 2/5Batch 300

([{(83.40799713134766, 0.8287931488519693)},
  {(83.5199966430664, 0.830841816427097)},
  {(83.95199584960938, 0.835247385721201)},
  {(84.23999786376953, 0.8385139322752947)},
  {(84.31999969482422, 0.8379940743056159)}],
 [{(83.32799530029297, 0.8280974588752512)},
  {(83.9679946899414, 0.8355230857504318)},
  {(84.2719955444336, 0.8378761983499856)},
  {(83.76000213623047, 0.8341326129438777)},
  {(84.60800170898438, 0.8397523455152802)}],
 [0.3372415368772611,
  0.32702942187468753,
  0.318907448329006,
  0.30303007299430135,
  0.2964557493963746],
 [0.4056741069166028,
  0.4095482724357624,
  0.4072017925126212,
  0.3974202989923711,
  0.3976640076357491])

In [ ]:
train_val_loop_test(train_loader_short, val_loader_short, test_loader_short)

Epoch: 1/5Batch 0/586Loss: 0.6931326985359192
Epoch: 1/5Batch 50/586Loss: 0.6891793608665466
Epoch: 1/5Batch 100/586Loss: 0.6736785173416138
Epoch: 1/5Batch 150/586Loss: 0.5976961255073547
Epoch: 1/5Batch 200/586Loss: 0.5359525084495544
Epoch: 1/5Batch 250/586Loss: 0.44365987181663513
Epoch: 1/5Batch 300/586Loss: 0.5402395129203796
Epoch: 1/5Batch 350/586Loss: 0.49689117074012756
Epoch: 1/5Batch 400/586Loss: 0.4443376660346985
Epoch: 1/5Batch 450/586Loss: 0.363057941198349
Epoch: 1/5Batch 500/586Loss: 0.45249223709106445
Epoch: 1/5Batch 550/586Loss: 0.32229888439178467


<ipython-input-260-bb6b16c7e67c>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  logits = F.softmax(model(features))


training accuracy: 80.44%
valid accuracy: 77.23%
Epoch: 2/5Batch 0/586Loss: 0.3973460793495178
Epoch: 2/5Batch 50/586Loss: 0.3935859203338623
Epoch: 2/5Batch 100/586Loss: 0.38213157653808594
Epoch: 2/5Batch 150/586Loss: 0.302183598279953
Epoch: 2/5Batch 200/586Loss: 0.41224390268325806
Epoch: 2/5Batch 250/586Loss: 0.41908887028694153
Epoch: 2/5Batch 300/586Loss: 0.2709755301475525
Epoch: 2/5Batch 350/586Loss: 0.38980206847190857
Epoch: 2/5Batch 400/586Loss: 0.360057532787323
Epoch: 2/5Batch 450/586Loss: 0.29168763756752014
Epoch: 2/5Batch 500/586Loss: 0.365852415561676
Epoch: 2/5Batch 550/586Loss: 0.43530890345573425
training accuracy: 90.05%
valid accuracy: 83.95%
Epoch: 3/5Batch 0/586Loss: 0.20116202533245087
Epoch: 3/5Batch 50/586Loss: 0.1717316210269928
Epoch: 3/5Batch 100/586Loss: 0.22574712336063385
Epoch: 3/5Batch 150/586Loss: 0.31779220700263977
Epoch: 3/5Batch 200/586Loss: 0.25272613763809204
Epoch: 3/5Batch 250/586Loss: 0.24847163259983063
Epoch: 3/5Batch 300/586Loss: 0.16612

In [ ]:
train_val_loop_test(train_loader_long, val_loader_long, test_loader_long)

Epoch: 1/15Batch 0/586Loss: 0.6312860250473022
Epoch: 1/15Batch 50/586Loss: 3.78112389398666e-07
Epoch: 1/15Batch 100/586Loss: 0.0
Epoch: 1/15Batch 150/586Loss: 0.0
Epoch: 1/15Batch 200/586Loss: 0.0
Epoch: 1/15Batch 250/586Loss: 0.0
Epoch: 1/15Batch 300/586Loss: 0.0024969736114144325
Epoch: 1/15Batch 350/586Loss: 2.9243426524772076e-07
Epoch: 1/15Batch 400/586Loss: 2.4864978058758425e-06
Epoch: 1/15Batch 450/586Loss: 8.772855153438286e-07
Epoch: 1/15Batch 500/586Loss: 4.656610741449185e-08
Epoch: 1/15Batch 550/586Loss: 2.738073590080603e-07
training accuracy: 50.00%
valid accuracy: 50.00%
Epoch: 2/15Batch 0/586Loss: 20.41591453552246
Epoch: 2/15Batch 50/586Loss: 0.0
Epoch: 2/15Batch 100/586Loss: 0.0
Epoch: 2/15Batch 150/586Loss: 1.8626450382086546e-09
Epoch: 2/15Batch 200/586Loss: 0.0
Epoch: 2/15Batch 250/586Loss: 0.0
Epoch: 2/15Batch 300/586Loss: 3.494354486465454
Epoch: 2/15Batch 350/586Loss: 9.804492037801538e-06
Epoch: 2/15Batch 400/586Loss: 7.94948027760256e-06
Epoch: 2/15Batch 45